In [1]:
import pandas as pd
import numpy as ny
import json
import nltk
import re
from collections import Counter
import matplotlib.pyplot as plt
#needed

In [ ]:
#Dictionary List
#Task 1: Transmissions, Incubation, and Environmental Stability
trans_Incub_enviro = ['transmissible','transmission','immune','infectious','symptomatic','airborne','spread','aerosol','aerosols']
#Task 2: COVID-19 Risk Factors
risk_factors = ['cancer','pregnant','pregnancy','death','critical','infection','diarrhea','risks','aneurysms','pneumonia','respiratory','mortality','severe','deaths','asthma','diabetes','severity','pneumonia']
#Task 3: Vaccines and Therapeutics
vacc = ['vaccination','antibody','vaccine','drug','hydroxychloroquine','drugs','vaccines''medicine''treated']
#Task 4: Virus genetics, orgin, and evolution
gene_ori_evo = ['genome','china','outbreak','genomes','mutations','source','origin','genomic','genes','strain','evolution','genetic','wuhan','protein','proteins']
#Task 5: Medical Care
med_care = ['treatment','antibody','surgical','prevention','antibodies','surgery''treated']
#Task 6: Ethical and social science considerations
Eth_soc = ['ethics','ethical','social','considerations','infants']
#Task 7: Non-pharmaceutical interventions
Non_pharma = ['therapies','telemedicine','isolation','protection','lockdown','treated','distancing','quarantine']
#Task 8: Diagnostics and Surveillance
diagno_surve = ['analysis','model','trial','surveillance','diagnostic','diagnosis','screening','monitoring','identification','imaging','testing','prognosis','estimation','predicting','modelling','detection','study','research']

In [ ]:
class Reader:
    def OpenFile(index, path):
        with open(path) as jsonF:
            content = json.load(jsonF)
            index.body_text = []
            for entry in content['body_text']:
                index.body_text.append(entry['text'])
            index.body_text = '\n'.join(index.body_text)
row = Reader(all_json[0])
print(row)

In [ ]:
#Functions
def getWords(index):
    Abstract = smallab.iloc[index]['abstract']
    #Abstract_words = re.split('(.*?\s.*?\s.*?\s.*?)\s', Abstract)
    Abstract_words = re.split('\\W+', Abstract)
    Abstract_words = list(map(lambda x: x.lower(), Abstract_words))
    return Abstract_words
#a = getWords(9)
#acleaned = remove(a)
#acleaned

def getWordsJ(index):
    J_list = journals.iloc[index]['journal']
    J_words = re.split('(.*?\s.*?\s.*?\s.*?)\s', J_list)
    J_words = list(map(lambda x: x.lower(), J_words))
    return J_words

def getWordsT(index):
    Ttles = smallt.iloc[index]['title']
    #titlew = re.split('(.*?\s.*?)\s', Ttles)
    titlew = re.split('\\W+', Ttles)
    titlew = list(map(lambda x: x.lower(), titlew))
    return titlew

def remove(List):
    newlist=[]
    for word in range(len(List)):
        if len(List[word]) >= 5:
            newlist.append(List[word])
    return newlist

def removeJ(List):
    newlist=[]
    for word in range(len(List)):
        newlist.append(List[word])
    return newlist

# Now I am going to run through and do a check for each Dictionary
def checkDict(List,Words):
    matches = []
    for i in Words:
        count = 0 
        for j in set(i):
            for k in List:
                if j == k:
                    count = count + 1
        matches.append(count)
    return matches

def insertTask(task,List):
    percentsim = []
    for i in task:
        temp = round((i/len(List))*100,1)
        percentsim.append(temp)
    return percentsim

In [2]:
dataF = pd.read_csv("metadata.csv", low_memory = False) 
#loading in the file with pandas instead because spark was having a problem withe column names
#needed
dataF

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158942,mixxjvte,2166cf20184f1b770a1a6f75682052fcfa42b396,Medline; PMC,Setting out on a fantastic voyage to advance n...,10.1038/s42003-020-0943-z,PMC7192934,32355225.0,cc-by,"We are inviting submissions of articles, persp...",2020-04-30,NaN,Commun Biol,NaN,NaN,NaN,document_parses/pdf_json/2166cf20184f1b770a1a6...,document_parses/pmc_json/PMC7192934.xml.json,https://doi.org/10.1038/s42003-020-0943-z; htt...,216650000.0
158943,t9bt70f2,d2a557f18ca3ad25962a7fce457eff2802f2adaa; a051...,BioRxiv; MedRxiv,Host and infectivity prediction of Wuhan 2019 ...,10.1101/2020.01.21.914044,NaN,NaN,medrxiv,"The recent outbreak of pneumonia in Wuhan, Chi...",2020-01-24,"Guo, Qian; Li, Mo; Wang, Chunhui; Wang, Peihon...",bioRxiv,NaN,NaN,NaN,document_parses/pdf_json/d2a557f18ca3ad25962a7...,NaN,https://doi.org/10.1101/2020.01.21.914044,213133457.0
158944,nhpjvgt4,da516a5f665fcb8a6f3a1fd350bcfa34820d951a,Medline; PMC,Considerations for Safety in the Use of System...,10.1111/dth.13687,PMC7283778,32458536.0,no-cc,"Coronavirus disease 2019 (COVID‐19), is respon...",2020-05-27,"Ricardo, Jose W.; Lipner, Shari R.",Dermatol Ther,NaN,NaN,NaN,document_parses/pdf_json/da516a5f665fcb8a6f3a1...,NaN,https://doi.org/10.1111/dth.13687; https://www...,218911875.0
158945,8gjng3o9,926208e67493a6e5ae8d770cae1fe98f3116f678,Elsevier; Medline; PMC,Wearing masks and the fight against the novel ...,10.1016/j.pulmoe.2020.05.011,PMC7256497,32507498.0,no-cc,NaN,2020-05-29,"Alizargar, Javad",Pulmonology,NaN,NaN,NaN,document_parses/pdf_json/926208e67493a6e5ae8d7...,document_parses/pmc_json/PMC7256497.xml.json,https://www.sciencedirect.com/science/article/...,218973065.0


In [ ]:
journals = dataF[['journal']]
#def getWords(index):
    journal_names = journals.iloc[index]['journal']
    journal_words = re.split('\\W+', journal_names)
    journal_words = list(map(lambda x: x.lower(), journal_words))
    return journal_words


In [ ]:
jour = []
for line in range(len(journals)):
    c = getWords(line)
    jour.append(c)
jour

In [ ]:
dataF['journal'].Counter().most_common()

In [ ]:
frequent = Counter(dataF[['journal']])
frequent.most_common()

In [ ]:
df = dataF.dropna(subset=['journal'])
df['journal'].value_counts().max()

In [ ]:
#not done coding
i =0 
columnname = 'pmc_json_files'
for i in pmc_jsonF.range(i,len(pmc_jsonF)):
    with open(pmc_jsonF['pmc_json_files'].values[i])

In [3]:
ab = dataF[['abstract']]
ab = ab.dropna()
ab.reset_index(drop=True,inplace=True)
titles = dataF[['title']]
titles = titles.dropna()
titles.reset_index(drop=True,inplace=True)
#journals = dataF[['journal']]
#journals = journals.dropna()
#journals.reset_index(drop=True,inplace=True)

In [4]:
def getWords(index):
    Abstract = smallab.iloc[index]['abstract']
    Abstract_words = re.split('\\W+', Abstract)
    Abstract_words = list(map(lambda x: x.lower(), Abstract_words))
    return Abstract_words


In [7]:
smallab = ab.iloc[0:117125]
cleanedablist = []
for line in range(len(smallab)):
    a = getWords(line)
    acleaned = remove(a)
    cleanedablist.append(acleaned)

In [ ]:
smallt = titles.iloc[0:158897]
cleanedtlist = []
for line in range(len(smallt)):
    t = getWordsT(line)
    tcleaned = remove(t)
    cleanedtlist.append(tcleaned)

In [ ]:
smallj = journals.iloc[0:150007]
cleanedjlist = []
for line in range(len(smallj)):
    j = getWordsJ(line)
    jcleaned = remove(j)
    cleanedjlist.append(jcleaned)

In [ ]:
Countj = Counter()
for i in cleanedjlist:
    for j in set(i):
        Countj[j] += 1
Countj.most_common()

In [ ]:
Count = Counter()
for i in cleanedablist:
    for j in set(i):
        Count[j] += 1
Count.most_common()

In [ ]:
Countt = Counter()
for i in cleanedtlist:
    for j in set(i):
        Countt[j] += 1
Countt.most_common()

In [ ]:
mostcommont = pd.DataFrame(Countt.most_common(30), columns=['words', 'count'])
fig, ax = plt.subplots(figsize=(8, 8))
mostcommont.sort_values(by='count').plot.barh(x='words',y='count', ax=ax, color="purple")
plt.show()

In [ ]:
mostcommon = pd.DataFrame(Count.most_common(30), columns=['words', 'count'])
fig, ax = plt.subplots(figsize=(8, 8))
mostcommon.sort_values(by='count').plot.barh(x='words',y='count', ax=ax, color="purple")
plt.show()

In [ ]:
mostcommonj = pd.DataFrame(Countj.most_common(30), columns=['words', 'count'])
fig, ax = plt.subplots(figsize=(8, 8))
mostcommonj.sort_values(by='count').plot.barh(x='words',y='count', ax=ax, color="purple")
plt.show()

In [9]:
#Creating a new DataFrame with the lists of words and Id of paper
Ab_ids = dataF[['cord_uid','abstract']]
Ab_ids = Ab_ids.dropna()
Ab_ids.reset_index(drop=True,inplace=True)
Ab_ids['abstract words'] = cleanedablist
Ab_ids

,cord_uid,abstract,abstract words
0,ug7v899j,OBJECTIVE: This retrospective chart review des...,"[objective, retrospective, chart, review, desc..."
1,02tnwd4m,Inflammatory diseases of the respiratory tract...,"[inflammatory, diseases, respiratory, tract, c..."
2,ejv2xln0,Surfactant protein-D (SP-D) participates in th...,"[surfactant, protein, participates, innate, re..."
3,2b73a28n,Endothelin-1 (ET-1) is a 21 amino acid peptide...,"[endothelin, amino, peptide, diverse, biologic..."
4,9785vg6d,Respiratory syncytial virus (RSV) and pneumoni...,"[respiratory, syncytial, virus, pneumonia, vir..."
...,...,...,...
117120,ntropr17,The seven NIH‐sponsored Regional Primate Resea...,"[seven, sponsored, regional, primate, research..."
117121,mixxjvte,"We are inviting submissions of articles, persp...","[inviting, submissions, articles, perspectives..."
117122,t9bt70f2,"The recent outbreak of pneumonia in Wuhan, Chi...","[recent, outbreak, pneumonia, wuhan, china, ca..."
117123,nhpjvgt4,"Coronavirus disease 2019 (COVID‐19), is respon...","[coronavirus, disease, covid, responsible, lea..."


In [12]:
task1 = checkDict(trans_Incub_enviro,cleanedablist)
pertask1=insertTask(task1,trans_Incub_enviro)

task2 = checkDict(risk_factors,cleanedablist)
pertask2=insertTask(task2,risk_factors)

task3 = checkDict(vacc,cleanedablist)
pertask3=insertTask(task3,vacc)

task4 = checkDict(gene_ori_evo,cleanedablist)
pertask4=insertTask(task4,gene_ori_evo)

task5 = checkDict(med_care,cleanedablist)
pertask5=insertTask(task5,med_care)

task6 = checkDict(Eth_soc,cleanedablist)
pertask6=insertTask(task6,Eth_soc)

task7 = checkDict(Non_pharma,cleanedablist)
pertask7=insertTask(task7,Non_pharma)

task8 = checkDict(diagno_surve,cleanedablist)
pertask8=insertTask(task8,diagno_surve)


In [17]:
Ab_ids['Transmissions, Incubation, and Environmental Stability'] = pertask1
Ab_ids['Risk Factors'] = pertask2
Ab_ids['Vaccines and Therapeutics'] = pertask3
Ab_ids['Virus genetics, orgin, and evolution'] = pertask4
Ab_ids['Medical Care'] = pertask5
Ab_ids['Ethical and social science considerations'] = pertask6
Ab_ids['Non-pharmaceutical interventions'] = pertask7
Ab_ids['Diagnostics and Surveillance'] = pertask8
Ab_ids.to_excel('output.xlsx', index=False)

In [ ]:
Counttask1 = Counter()
for i in percentsim:
    Counttask1[i] += 1
Counttask1.most_common()

In [ ]:

Task1df = pd.DataFrame(Counttask1.most_common()[-6:], columns=['percent', 'count'])
fig, ax = plt.subplots(figsize=(8, 8))
Task1df.sort_values(by='percent').plot.bar(x='percent',y='count', ax=ax, color="blue")
plt.xlabel('Similarity Score', color = 'white')
plt.ylabel('Number of Articles', color = 'white')
plt.xticks(color = 'white')
plt.yticks(color = 'white')
plt.title('Number of Articles and How They Relate to Transmissions, Incubation, and Environmental Stability of COVID-19', color = 'white')
plt.show()

In [ ]:
Task1best = Ab_ids.loc[Ab_ids['Task 1 percent'] == 66.7]
Task1best.reset_index(drop=True,inplace=True)
Task1best.iloc[0]['abstract']